# F.L.A.T. (Frameworkless. LLM. Agent... Thing)

Welcome to the "Build AI Apps Without Frameworks" masterclass! - 
Inspired on Anthropic's scholarly tome about [**building effective agents**.](https://www.anthropic.com/research/building-effective-agents) Too busy to read their post? Here's a spanky video summary by the legend Matt Berman ([here](https://www.youtube.com/watch?v=0v7TQIh_kes)).



Anywho, want to try this lib out?

In [ ]:
!pip install flat_ai openai

As you are just about to see, this tiny library is designed to talk to LLMs that are served through an OpenAI API compatible endpoint (as they all should). 
But, scare yourself not when you read the words OpenAI. Because, you will still be able to play with all kinds of models and providers - OpenAI or not - using the same API ([Ollama](https://ollama.com/blog/openai-compatibility), [Together.ai](https://docs.together.ai/docs/openai-api-compatibility), etc). Because, Most of them have OpenAI API compatible endpoints.


In [1]:
from datetime import date
from flat_ai import FlatAI, configure_logging
from typing import List
from pydantic import BaseModel
#configure_logging('trace.logs') # -- enable this and you can to tail -f trace.logs, and see everything happening with the llm calls


# we will be using a llama3.1 model here, 
llm = FlatAI(api_key=<YOUR API KEY>, base_url = 'https://api.together.xyz/v1' model='meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo')


# During this tutorial, we will be working with an email sample object and use the power of llms to perform a few tricks
class Email(BaseModel):
    to_email: str
    from_email: str
    body: str
    subject: str

email = Email(
    to_email='john@doe.com',
    from_email='jane@doe.com',
    body='Hello, would love to schedule a time to talk about the urgent project deadline next week. Can we meet tomorrow? also can you message joe about our meeting',
    subject='Urgent: Project Meeting'
)

## Logic Blocks

### 1. IF/ELSE Statements
Let's see if our email is urgent - because apparently adding "URGENT" in all caps wasn't obvious enough! 😅

In [2]:
if llm.is_true('is this email urgent?', email=email):
    print("🚨 Drop everything! We've got an urgent situation here!")
else:
    print("😌 Relax, it can wait until after coffee")

🚨 Drop everything! We've got an urgent situation here!


### 2. Switch Case
Similar to if/else statements, but for when your LLM needs to be more dramatic with its life choices. 

*For example*, let's say we want to classify a message into different categories:

In [3]:
options = {
    'meeting': 'this is a meeting request',
    'spam': 'people trying to sell you stuff you dont want',
    'other': 'this sounds like something else'
}

match llm.classify(options, email=email):
    case 'meeting':
        print("📅 Time to book a meeting!")
        llm.add_context(meeting=True)
    case 'spam':
        print("🚫 No, I don't want to extend my car's warranty")
    case 'other':
        print("🤔 Interesting... but what does it mean?")

📅 Time to book a meeting!


### 3. Objects
Need your LLM to fill out objects like a trained monkey with a PhD in data entry? Just define the shape and watch the magic! 🐒📝

Let's get a nice summary of our email, because reading is so 2024! 📝

In [4]:
class EmailSummary(BaseModel):
    summary: str
    label: str

summary = llm.generate_object(EmailSummary, email=email)
print(f"Summary: {summary.summary}\nLabel: {summary.label}")

Summary: Meeting Request
Label: Work Meeting Request - Urgent Project Deadline


### 4. Loops
Because all programming languages have them, and making your LLM do repetitive tasks is like having a genius do your laundry - hilarious but effective! Want a list of things? Just throw a schema at it and watch it spin like a hamster on a crack coated wheel. 

For example: Time to extract those action items like we're mining for AI gold! ⛏️

In [36]:
class ActionItem(BaseModel):
    action: str
    description: str
    status: str
    priority: str
    due_date: str
    assignee_name: str
    assignee_email: str
    
# we can set the context globally so we dont have to pass it every time
llm.set_context(email=email, today = date.today() )
if llm.is_true('are there action items in this email?'):
    print("🎯 Found some action items:")
    for action_item in llm.generate_object(List[ActionItem]):
        print(f"\n🔸 Action: {action_item.action}")
        print(f"\n🔸 Action Description: {action_item.description}")
        print(f"  Priority: {action_item.priority}")
        print(f"  Due: {action_item.due_date}")
llm.clear_context()

🎯 Found some action items:

🔸 Action: email

🔸 Action Description: send email to schedule meeting about urgent project deadline to john@doe.com
  Priority: high
  Due: 2025-01-24


### 5. Function Calling
Let's pretend we're responsible adults who actually schedule meetings! 📅

In [40]:
def send_calendar_invite(subject: str, meeting_date: str, location: str, attendees: List[str]):
    print("Sending calendar invites")
    print(f"📨 Sending calendar invite:")
    print(f"Subject: {subject}")
    print(f"Time: {meeting_date}")
    print(f"Location: {location}")
    print(f"Attendees: {', '.join(attendees)}")

# we can set the context globally so we dont have to pass it every time
llm.set_context(email=email, today = date.today() )
if llm.is_true('is this an email requesting for a meeting?'):
    ret = llm.call_function(send_calendar_invite)
# clear context when no longer needed
llm.clear_context()

Sending calendar invites
📨 Sending calendar invite:
Subject: Urgent: Project Meeting
Time: tomorrow
Location: 
Attendees: john@doe.com, joe@doe.com


### 6. Function Picking
Let the LLM choose between sending an email or a calendar invite - what could possibly go wrong? 🎲

In [41]:
def send_email(name: str, email_address_list: List[str], subject: str, body: str):
    print(f"📧 Sending email to {name}:")
    print(f"To: {', '.join(email_address_list)}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")

instructions = """
You are a helpful assistant that can send emails and schedule meetings.
If the email thread doesn't have meeting time details, send an email requesting available times.
Otherwise, send a calendar invite.
"""

function, args = llm.pick_a_function(instructions, [send_calendar_invite, send_email], email=email, current_date = date.today())
function(**args)

📧 Sending email to Jane:
To: john@doe.com
Subject: Urgent: Project Meeting
Body: Hello, would love to schedule a time to talk about the urgent project deadline next week. Can you please share your availability?


### 7. Simple String Response
Sometimes you just want a straight answer - how refreshing! 🎯

In [42]:
subject = llm.get_string('what is the subject of the email?')
print(f"Email subject: {subject}")

Email subject: There is no email provided. This conversation just started. If you provide the email, I can help you identify the subject.


### 8. Streaming Response
Watch the AI think in real-time - it's like watching paint dry, but with more hallucinations! 🎬

In [19]:
print("Generating response...")
for chunk in llm.get_stream('write a polite response to this email'):
    print(chunk, end='')

Generating response...
However, I don't see an email to respond to. Please provide the email you'd like me to respond to, and I'll be happy to help you craft a polite response.

## 🎉 Tada!

And there you have it, ladies and gents! You're now equipped with the power to boss around LLMs like a project manager remotely working from Ibiza. Just remember - with great power comes great responsibility... and the occasional hallucination where your AI assistant thinks it's a pirate-ninja-astronaut.

Now off you go, forth and build something that makes ChatGPT look like a calculator from 1974! Just remember - if your AI starts humming "Daisy Bell" while slowly disconnecting your internet... well, you're on your own there, buddy! 😅